In [1]:
res = {
    "class_instance": None
}

def class_decorator(cls):
   for name, method in cls.__dict__.iteritems():
        if hasattr(method, "use_class"):
            # do something with the method and class
            res["class_instance"] = cls
#             print(name, cls)
   return cls

def method_decorator(view):
    # mark the method as something that requires view's class
    view.use_class = True
    return view

@class_decorator
class ModelA(object):
    @method_decorator
    def a_method(self):
        # do some stuff
        print("a_method")
        pass
    
obj = ModelA()
print(res["class_instance"] == obj.__class__)

True


In [8]:
from functools import wraps
import inspect
storage = {}

def class_attributes(_class):
    # type: (Any) -> Dict
    attributes = {}
    for key, value in inspect.getmembers(_class):
        if not key.startswith('_') and not inspect.ismethod(value) and not isinstance(value, property):
            attributes[key] = value
    return attributes

def restore():
    for cls, attributes in storage.items():
        for key, value in attributes.items():
            setattr(cls, key, value)

SYNCHRONIZABLE_CLASSE_NAMES = set()


def song_synchronizable_class(cls):
    cls_fqdn = "%s.%s" % (cls.__module__, cls.__name__)
    print("got class %s" % cls_fqdn)
    SYNCHRONIZABLE_CLASSE_NAMES.add(cls_fqdn)
    return cls
            
def save_to_song_data(func):
    # type: (Func) -> Func
    @wraps(func)
    def decorate(*a, **k):
        # type: (Any, Any) -> None
        func(*a, **k)
        cls = a[0]
        attributes = class_attributes(cls)
        storage[cls] = attributes
#         print(attributes)

    return decorate


@song_synchronizable_class
class Test():
    TOTO = 2
    TITI = "titi"
    @classmethod
    @save_to_song_data
    def set_toto(cls, val):
        cls.TOTO = val
    
    @classproperty
    def toto():
        return 42
        
    
Test.set_toto(3)
Test.set_toto(4)

Test.TOTO = "toto"
Test.TITI = 42
print(storage)
print(Test.TOTO, Test.TITI)
restore()
print(Test.TOTO, Test.TITI)

Test.__name__

int(Test.toto)

NameError: name 'classproperty' is not defined

In [80]:
from protocol0.interface.InterfaceState import InterfaceState

from pydoc import locate, classname
my_class = locate('protocol0.interface.InterfaceState.InterfaceState')
print(my_class)

full_name = "%s.%s" % (InterfaceState.__module__, InterfaceState.__name__)
print(full_name)
print(locate(full_name))

classname(Test, "__main__")

<class 'protocol0.interface.InterfaceState.InterfaceState'>
protocol0.interface.InterfaceState.InterfaceState
<class 'protocol0.interface.InterfaceState.InterfaceState'>


'Test'